In [1]:
from pyspark.sql import SparkSession

In [2]:
sparkSesison = (SparkSession.builder
                 .config("spark.cassandra.connection.host","cassandra")
                 .config("spark.cassandra.auth.username","cassandra")
                 .config("spark.cassandra.auth.password","cassandra")
                 .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1,com.datastax.spark:spark-cassandra-connector_2.12:3.0.0")
                 .appName("demo").getOrCreate()
                 )


:: loading settings :: url = jar:file:/usr/local/lib/python3.8/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-df751cf9-c61b-42db-a91e-01b7067d9edf;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.1 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
	found com.datastax.spark#spark-cassandra-connector_2.12;3.0.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2

In [3]:
from pyspark.sql import DataFrame

In [4]:
def dataFrameFromCassandaDbTable(sparkSession:SparkSession,keyspace:str,table:str)->DataFrame:
    df = (sparkSession.read
        .format("org.apache.spark.sql.cassandra")
        .options(table=table, keyspace=keyspace)
        .load())
    return df

In [5]:
df = dataFrameFromCassandaDbTable(sparkSession=sparkSesison,keyspace="ineuron",table="employee")

In [6]:
df.show()

+------+----------+--------+-------------+
|emp_id|      city|emp_name|        state|
+------+----------+--------+-------------+
|    30|     Delhi|    Emma|        Delhi|
|     2| Bengalore|  Avnish|    Karnataka|
|     5|   Chennai|   Berry|    Tamilnadu|
|    28| Hyderabad|   Akash|Andra Pradesh|
|     6|      Pune|  Barton|  Maharashtra|
|    13|     Noida|   Rahul|Uttar Pradesh|
|    16|    Nagpur|  Vikash|  Maharashtra|
|    23|    Nagpur|    Amit|  Maharashtra|
|    26|   Chennai|     Pia|    Tamilnadu|
|    21|     Delhi|  Shivam|        Delhi|
|    31|New Mumbai|Abhishek|  Maharashtra|
|     4|    Mumbai| Stephen|  Maharashtra|
|    18|      Pune|    Adam|  Maharashtra|
|    24| Hyderabad| Susmita|Andra Pradesh|
|    27|      Pune|   Ankit|  Maharashtra|
|    14|     Delhi|   Sunny|        Delhi|
|    20|     Noida|  Robert|Uttar Pradesh|
|    11|    Nagpur|   Krish|  Maharashtra|
|     1| Bengalore|  Avnish|    Karnataka|
|    25|    Mumbai|    Iris|  Maharashtra|
+------+---

In [7]:
df.count()

30

In [10]:
df.groupBy(['state','city']).count().show()

+-------------+----------+-----+
|        state|      city|count|
+-------------+----------+-----+
|Uttar Pradesh|     Noida|    4|
|  Maharashtra|New Mumbai|    4|
|    Tamilnadu|   Chennai|    3|
|  Maharashtra|    Mumbai|    2|
|  Maharashtra|      Pune|    3|
|Andra Pradesh| Hyderabad|    5|
|        Delhi|     Delhi|    4|
|  Maharashtra|    Nagpur|    3|
|    Karnataka| Bengalore|    2|
+-------------+----------+-----+



In [11]:
df.createOrReplaceTempView("emp")

In [13]:
sparkSesison.sql("select * from emp").show()

+------+----------+---------+-------------+
|emp_id|      city| emp_name|        state|
+------+----------+---------+-------------+
|    20|     Noida|   Robert|Uttar Pradesh|
|    26|   Chennai|      Pia|    Tamilnadu|
|    21|     Delhi|   Shivam|        Delhi|
|     6|      Pune|   Barton|  Maharashtra|
|    13|     Noida|    Rahul|Uttar Pradesh|
|    14|     Delhi|    Sunny|        Delhi|
|    28| Hyderabad|    Akash|Andra Pradesh|
|    25|    Mumbai|     Iris|  Maharashtra|
|    15|New Mumbai|   Vishal|  Maharashtra|
|     1| Bengalore|   Avnish|    Karnataka|
|     7| Hyderabad|  Natasha|Andra Pradesh|
|    30|     Delhi|     Emma|        Delhi|
|     2| Bengalore|   Avnish|    Karnataka|
|    11|    Nagpur|    Krish|  Maharashtra|
|     8|     Noida|   Sundar|Uttar Pradesh|
|    17|   Chennai|  Aravind|    Tamilnadu|
|    10|New Mumbai|Sudhanshu|  Maharashtra|
|    23|    Nagpur|     Amit|  Maharashtra|
|    29|     Noida|    Harry|Uttar Pradesh|
|    22|New Mumbai|   Deepak|  M

In [18]:
sparkSesison.sql("select * from emp where emp_name like 'S%t'").show()

+------+---------+--------+-------------+
|emp_id|     city|emp_name|        state|
+------+---------+--------+-------------+
|    19|Hyderabad|Scarlett|Andra Pradesh|
+------+---------+--------+-------------+



In [20]:
df.show()

+------+----------+---------+-------------+
|emp_id|      city| emp_name|        state|
+------+----------+---------+-------------+
|     7| Hyderabad|  Natasha|Andra Pradesh|
|     8|     Noida|   Sundar|Uttar Pradesh|
|     5|   Chennai|    Berry|    Tamilnadu|
|    30|     Delhi|     Emma|        Delhi|
|     2| Bengalore|   Avnish|    Karnataka|
|    11|    Nagpur|    Krish|  Maharashtra|
|    22|New Mumbai|   Deepak|  Maharashtra|
|    23|    Nagpur|     Amit|  Maharashtra|
|    28| Hyderabad|    Akash|Andra Pradesh|
|     6|      Pune|   Barton|  Maharashtra|
|    13|     Noida|    Rahul|Uttar Pradesh|
|    26|   Chennai|      Pia|    Tamilnadu|
|    21|     Delhi|   Shivam|        Delhi|
|    31|New Mumbai| Abhishek|  Maharashtra|
|    29|     Noida|    Harry|Uttar Pradesh|
|     1| Bengalore|   Avnish|    Karnataka|
|    17|   Chennai|  Aravind|    Tamilnadu|
|    10|New Mumbai|Sudhanshu|  Maharashtra|
|    14|     Delhi|    Sunny|        Delhi|
|     9|     Delhi| Shashank|   

In [21]:
from pyspark.sql.functions import *

In [22]:
from pyspark.sql.types import *

In [74]:
df  = df.select(col("emp_id").cast(StringType()).alias("key"),to_json(struct("emp_id","emp_name","city","state")).alias("value"))

In [39]:
df.printSchema()

root
 |-- key: string (nullable = false)
 |-- to_json(struct(emp_id, emp_name, city, state)): string (nullable = true)



In [40]:
#kafka server detail
KAFKA_BOOTSTRAP_SERVER="kafka:9092"
KAFKA_TOPIC = "employee"

In [72]:
df = (sparkSesison
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers",KAFKA_BOOTSTRAP_SERVER)
    .option("subscribe",KAFKA_TOPIC)
    .option("startingOffsets","earliest")
    .load()
    )

In [42]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [73]:
df = sparkSesison.read.parquet("/project/employee_data")

In [47]:
df.show()

+------+--------+----------+-------------+
|emp_id|emp_name|      city|        state|
+------+--------+----------+-------------+
|    31|Abhishek|New Mumbai|  Maharashtra|
|     8|  Sundar|     Noida|Uttar Pradesh|
|    27|   Ankit|      Pune|  Maharashtra|
|    30|    Emma|     Delhi|        Delhi|
|     7| Natasha| Hyderabad|Andra Pradesh|
|    19|Scarlett| Hyderabad|Andra Pradesh|
|     6|  Barton|      Pune|  Maharashtra|
|    13|   Rahul|     Noida|Uttar Pradesh|
|     5|   Berry|   Chennai|    Tamilnadu|
|    16|  Vikash|    Nagpur|  Maharashtra|
|    17| Aravind|   Chennai|    Tamilnadu|
|    25|    Iris|    Mumbai|  Maharashtra|
|    29|   Harry|     Noida|Uttar Pradesh|
|    15|  Vishal|New Mumbai|  Maharashtra|
|    11|   Krish|    Nagpur|  Maharashtra|
|    24| Susmita| Hyderabad|Andra Pradesh|
|     4| Stephen|    Mumbai|  Maharashtra|
|     2|  Avnish| Bengalore|    Karnataka|
|    18|    Adam|      Pune|  Maharashtra|
|    23|    Amit|    Nagpur|  Maharashtra|
+------+---

In [48]:
df.count()

180

In [75]:
df = df.select("key",encode(col("value"),charset="UTF-8").alias("value"))

In [76]:
df.show()

+---+--------------------+
|key|               value|
+---+--------------------+
| 31|[7B 22 65 6D 70 5...|
|  8|[7B 22 65 6D 70 5...|
| 27|[7B 22 65 6D 70 5...|
| 30|[7B 22 65 6D 70 5...|
|  7|[7B 22 65 6D 70 5...|
| 19|[7B 22 65 6D 70 5...|
|  6|[7B 22 65 6D 70 5...|
| 13|[7B 22 65 6D 70 5...|
|  5|[7B 22 65 6D 70 5...|
| 16|[7B 22 65 6D 70 5...|
| 17|[7B 22 65 6D 70 5...|
| 25|[7B 22 65 6D 70 5...|
| 29|[7B 22 65 6D 70 5...|
| 15|[7B 22 65 6D 70 5...|
| 11|[7B 22 65 6D 70 5...|
| 24|[7B 22 65 6D 70 5...|
|  4|[7B 22 65 6D 70 5...|
|  2|[7B 22 65 6D 70 5...|
| 18|[7B 22 65 6D 70 5...|
| 23|[7B 22 65 6D 70 5...|
+---+--------------------+
only showing top 20 rows



In [80]:

schema = StructType(fields=[

StructField(name="emp_id",dataType=IntegerType()),
StructField(name="emp_name",dataType=StringType()),
StructField(name="city",dataType=StringType()),
StructField(name="state",dataType=StringType()),
])

In [84]:
df.select(from_json(decode(col("value"),charset="UTF-8"),schema=schema).alias("value")).select("value.*").show(truncate=False)

+------+--------+----------+-------------+
|emp_id|emp_name|city      |state        |
+------+--------+----------+-------------+
|31    |Abhishek|New Mumbai|Maharashtra  |
|8     |Sundar  |Noida     |Uttar Pradesh|
|27    |Ankit   |Pune      |Maharashtra  |
|30    |Emma    |Delhi     |Delhi        |
|7     |Natasha |Hyderabad |Andra Pradesh|
|19    |Scarlett|Hyderabad |Andra Pradesh|
|6     |Barton  |Pune      |Maharashtra  |
|13    |Rahul   |Noida     |Uttar Pradesh|
|5     |Berry   |Chennai   |Tamilnadu    |
|16    |Vikash  |Nagpur    |Maharashtra  |
|17    |Aravind |Chennai   |Tamilnadu    |
|25    |Iris    |Mumbai    |Maharashtra  |
|29    |Harry   |Noida     |Uttar Pradesh|
|15    |Vishal  |New Mumbai|Maharashtra  |
|11    |Krish   |Nagpur    |Maharashtra  |
|24    |Susmita |Hyderabad |Andra Pradesh|
|4     |Stephen |Mumbai    |Maharashtra  |
|2     |Avnish  |Bengalore |Karnataka    |
|18    |Adam    |Pune      |Maharashtra  |
|23    |Amit    |Nagpur    |Maharashtra  |
+------+---

In [ ]:
(df.withColumn("value",
        from_json(decode("value",charset="UTF-8"),schema=schema)
        .alias("value"))
        .select("value.*")
    )